# Introduction

Do higher film budgets lead to more box office revenue? Let's find out if there's a relationship using the movie budgets and financial performance data that I've scraped from [the-numbers.com](https://www.the-numbers.com/movie/budgets) on **May 1st, 2018**. 

<img src=https://i.imgur.com/kq7hrEh.png>

# Import Statements

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

# Notebook Presentation

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

# Read the Data

In [ ]:
data = pd.read_csv('./data/cost_revenue_dirty.csv')

# Explore and Clean the Data

In [ ]:
data.head()

**Challenge**: Answer these questions about the dataset:
1. How many rows and columns does the dataset contain?
2. Are there any NaN values present?
3. Are there any duplicate rows?
4. What are the data types of the columns?

In [ ]:
print(f"Dataset has {data.shape[0]} rows and {data.shape[1]} columns.")
print(f"Does dataset has NaN Values? [{data.isna().values.any()}]")
print(f"Does dataset have duplicate rows? [{data.duplicated().values.any()}]")
print(f"Datatype of columns:\n{data.dtypes}")

### Data Type Conversions

**Challenge**: Convert the `USD_Production_Budget`, `USD_Worldwide_Gross`, and `USD_Domestic_Gross` columns to a numeric format by removing `$` signs and `,`. 
<br>
<br>
Note that *domestic* in this context refers to the United States.

In [ ]:
# Cleaning
data["USD_Production_Budget"] = data["USD_Production_Budget"].astype(str).str.replace("$", "").str.replace(",", "")
data["USD_Worldwide_Gross"] = data["USD_Worldwide_Gross"].astype(str).str.replace("$", "").str.replace(",", "")
data["USD_Domestic_Gross"] = data["USD_Worldwide_Gross"].astype(str).str.replace("$", "").str.replace(",", "")

# Conversions
data["USD_Production_Budget"] = pd.to_numeric(data["USD_Production_Budget"])
data["USD_Worldwide_Gross"] = pd.to_numeric(data["USD_Worldwide_Gross"])
data["USD_Domestic_Gross"] = pd.to_numeric(data["USD_Domestic_Gross"])

data.dtypes

**Challenge**: Convert the `Release_Date` column to a Pandas Datetime type. 

In [ ]:
data["Release_Date"] = pd.to_datetime(data["Release_Date"])
data.dtypes

### Descriptive Statistics

**Challenge**: 

1. What is the average production budget of the films in the data set?
2. What is the average worldwide gross revenue of films?
3. What were the minimums for worldwide and domestic revenue?
4. Are the bottom 25% of films actually profitable or do they lose money?
5. What are the highest production budget and highest worldwide gross revenue of any film?
6. How much revenue did the lowest and highest budget films make?

In [ ]:
data.describe().T

# Investigating the Zero Revenue Films

**Challenge** How many films grossed $0 domestically (i.e., in the United States)? What were the highest budget films that grossed nothing?

In [ ]:
zero_domestic: pd.DataFrame = data[data["USD_Domestic_Gross"] == 0]
zero_domestic = zero_domestic.sort_values(by="USD_Production_Budget", ascending=False)
print(f"Number of films that made 0$ domestically: {len(zero_domestic)}")
zero_domestic

**Challenge**: How many films grossed $0 worldwide? What are the highest budget films that had no revenue internationally?

In [ ]:
zero_worldwide: pd.DataFrame = data[data["USD_Worldwide_Gross"] == 0]
zero_worldwide = zero_worldwide.sort_values(by="USD_Production_Budget", ascending=False)
print(f"Number of films that made 0$ worldwide: {len(zero_worldwide)}")
zero_worldwide

### Filtering on Multiple Conditions

So far, we've created subsets for our DataFrames based on a single condition. But what if we want to select our data based on more than one condition? For example, which films made money internationally (i.e., data.USD_Worldwide_Gross != 0), but had zero box office revenue in the United States (i.e., data.USD_Domestic_Gross == 0)? 

How would we create a filter for these two conditions? One approach is to use the `.loc[]` property combined with the [bitwise](https://docs.python.org/3.4/library/operator.html#mapping-operators-to-functions) and `&` operator.

In [ ]:
only_zero_domestically = data.loc[(data["USD_Domestic_Gross"] == 0) & (data["USD_Worldwide_Gross"] != 0)]
only_zero_domestically

**Challenge**: Use the [`.query()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html) to accomplish the same thing. Create a subset for international releases that had some worldwide gross revenue, but made zero revenue in the United States. 

Hint: This time you'll have to use the `and` keyword.

In [ ]:
only_zero_domestically_query = data.query("USD_Worldwide_Gross != 0 and USD_Domestic_Gross == 0")
only_zero_domestically_query

### Unreleased Films

**Challenge**:
* Identify which films were not released yet as of the time of data collection (May 1st, 2018).
* How many films are included in the dataset that have not yet had a chance to be screened in the box office? 
* Create another DataFrame called data_clean that does not include these films. 

In [ ]:
# Date of Data Collection
scrape_date = pd.Timestamp('2018-5-1')
future_releases = data[data["Release_Date"] > scrape_date]
future_releases

In [ ]:
# We will work on clean dataframe
data_cleaned = data.drop(future_releases.index)
data_cleaned.shape

### Films that Lost Money

**Challenge**: 
What is the percentage of films where the production costs exceeded the worldwide gross revenue? 

In [ ]:
data_cleaned.describe()

In [ ]:
budget_over_worldwide_gross = data_cleaned[data_cleaned["USD_Production_Budget"] > data_cleaned["USD_Worldwide_Gross"]]
print(f"Number of films that there budget exceed the revenue: {len(budget_over_worldwide_gross)}")
print(f"Percentage: {len(budget_over_worldwide_gross) / len(data_cleaned)}")
budget_over_worldwide_gross

# Seaborn for Data Viz: Bubble Charts

In [ ]:
# Seaborn is built on top of Matplotlib

plt.figure(figsize=(8, 4), dpi=200)

fig = sns.scatterplot(
    data=data_cleaned,
    x="USD_Production_Budget",
    y="USD_Worldwide_Gross"
)

fig.set(
    ylim=(0, 3000000000),
    xlim=(0, 450000000),
    xlabel="Budget in $100 Millions",
    ylabel="Revenue in $ Billions"
)

plt.show()

Seaborn Built on top of Matplotlib
<br>
<img src="./images/scatter_returns.png">

### Plotting Movie Releases over Time

**Challenge**: Try to create the following Bubble Chart:

<img src=https://i.imgur.com/8fUn9T6.png>



In [ ]:
plt.figure(figsize=(8, 4), dpi=200)

fig_bubble = sns.scatterplot(
    data=data_cleaned,
    x="Release_Date",
    y="USD_Production_Budget",
    hue="USD_Worldwide_Gross"
)

fig_bubble.set(
    xlim=(data_cleaned.Release_Date.min(), data_cleaned.Release_Date.max()),
    ylim=(0, 450000000),
    xlabel="Year",
    ylabel="Revenue in $100 Millions"
)

plt.show()  # Small budgets = few films, More releases = Growing budgets

# Converting Years to Decades Trick

**Challenge**: Create a column in `data_clean` that has the decade of the release. 

<img src=https://i.imgur.com/0VEfagw.png width=650> 

Here's how: 
1. Create a [`DatetimeIndex` object](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html) from the Release_Date column. 
2. Grab all the years from the `DatetimeIndex` object using the `.year` property.
<img src=https://i.imgur.com/5m06Ach.png width=650>
3. <br>
4. Use floor division `//` to convert the year data to the decades of the films.
5. Add the decades as a `Decade` column to the `data_clean` DataFrame.

### pd.DatetimeIndex:
[Source](https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.html)

In [ ]:
dt_index = pd.DatetimeIndex(data_cleaned["Release_Date"])
years = dt_index.year
decades = (years // 10) * 10
data_cleaned["Decade"] = decades
# OR: years - (years % 10)
data_cleaned.tail()

### Separate the "old" (before 1969) and "New" (1970s onwards) Films

**Challenge**: Create two new DataFrames: `old_films` and `new_films`
* `old_films` should include all the films before 1969 (up to and including 1969)
* `new_films` should include all the films from 1970 onwards
* How many films were released prior to 1970?
* What was the most expensive film made prior to 1970?

In [ ]:
old_films = data_cleaned[data_cleaned.Decade <= 1969]
new_films = data_cleaned[data_cleaned.Decade > 1970]
old_films.describe()

In [ ]:
old_films.sort_values(by="USD_Production_Budget", ascending=False)  # cleopatra was most expensive

In [ ]:
new_films.describe()

# Seaborn Regression Plots

Let's visualise the relationship between the movie budget and the worldwide revenue using linear regression. Seaborn makes this incredibly easy with the `.regplot()` function.

In [ ]:
plt.figure(figsize=(8, 4), dpi=200)

with sns.axes_style("whitegrid"):
    sns.regplot(
        old_films,
        x="USD_Production_Budget",
        y="USD_Worldwide_Gross",
        scatter_kws={"alpha": 0.4},
        line_kws={"color": "black"}
    )

plt.show()

**Challenge**: Use Seaborn's `.regplot()` to show the scatter plot and linear regression line against the `new_films`. 
<br>
<br>
Style the chart

* Put the chart on a `'darkgrid'`.
* Set limits on the axes so that they don't show negative values.
* Label the axes on the plot "Revenue in  billions" and "Budget in  millions".
* Provide HEX colour codes for the plot and the regression line. Make the dots dark blue (#2f4b7c) and the line orange (#ff7c43).

Interpret the chart

* Do our data points for the new films align better or worse with the linear regression than for our older films?
* Roughly how much would a film with a budget of $150 million make according to the regression line?

In [ ]:
plt.figure(figsize=(8,4), dpi=200)
with sns.axes_style('darkgrid'):
  ax = sns.regplot(data=new_films,
                   x='USD_Production_Budget',
                   y='USD_Worldwide_Gross',
                   color='#2f4b7c',
                   scatter_kws = {'alpha': 0.3},
                   line_kws = {'color': '#ff7c43'})
  
  ax.set(ylim=(0, 3000000000),
         xlim=(0, 450000000),
         ylabel='Revenue in $ billions',
         xlabel='Budget in $100 millions') 
  
plt.show()

# Run Your Own Regression with scikit-learn (Skipped)

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**: Run a linear regression for the `old_films`. Calculate the intercept, slope and r-squared. How much of the variance in movie revenue does the linear model explain in this case?

# Use Your Model to Make a Prediction

We just estimated the slope and intercept! Remember that our Linear Model has the following form:

$$ REV \hat ENUE = \theta _0 + \theta _1 BUDGET$$

**Challenge**:  How much global revenue does our model estimate for a film with a budget of $350 million? 